In [4]:
conda install -c conda-forge geopandas


Note: you may need to restart the kernel to use updated packages.


In [2]:
import plotly

import json
import numpy as np
import pandas as pd

import plotly.express as px

# import geopandas as gpd

In [21]:
## plotting plotly in browser

# import plotly.io as pio
# pio.renderers.default = 'browser'

In [3]:
print(plotly.__version__)

4.13.0


In [3]:
# print(geopandas.__version__)

In [40]:
##. 1 GeoJSON
indo_states = json.load(open('indo_geo.json', 'r'))

In [36]:
# for i in indo_states['features']:
#     print(i['properties'])
indo_states['features'][0]['properties']

{'Shape_Leng': 27.3673635506,
 'Shape_Area': 4.62543727674,
 'ADM1_EN': 'Aceh',
 'ADM1_PCODE': 'ID11',
 'ADM1_REF': None,
 'ADM1ALT1EN': None,
 'ADM1ALT2EN': None,
 'ADM0_EN': 'Indonesia',
 'ADM0_PCODE': 'ID',
 'date': '2019-12-20',
 'validOn': '2020-04-01',
 'validTo': None}

In [20]:
indo_states_map = {}

for state in indo_states['features']:
    state['id'] = state['properties']["ADM1_PCODE"]
    indo_states_map[state['properties']['ADM1_EN']] = state['id']

# indo_states_map

In [18]:
## 2. get covid data on province level

df = pd.read_csv('indo-cases.csv')
df.head()

## adjust names in df to match names in indo_states_map
df = df.replace('Bangka Belitung', 'Kepulauan Bangka Belitung')
df = df.replace("DKI Jakarta", 'Dki Jakarta')

## drop row 'Indonesia' 
df = df.drop(index=34)
## add column id with 'ids' from indo-state-map list
df['ADM1_PCODE'] = df['Provinsi'].apply(lambda x: indo_states_map[x])
df.head()


,X,Y,FID,Kode_Provi,Provinsi,Kasus_Posi,Kasus_Semb,Kasus_Meni,ADM1_PCODE
0,96.909523,4.225082,1,11,Aceh,8199,6550,311,ID11
1,99.057888,2.184779,2,12,Sumatera Utara,13277,10846,542,ID12
2,100.466805,-0.851914,3,13,Sumatera Barat,14940,10764,270,ID13
3,101.812892,0.510526,4,14,Riau,14841,11490,340,ID14
4,103.599320,-1.617750,5,15,Jambi,1735,1256,36,ID15


In [44]:
# Step 3 : Visualization

## Option 1 : using featureidkey

fig = px.choropleth(df, geojson=indo_states, color="Kasus_Posi",
                    locations="ADM1_PCODE", featureidkey="properties.ADM1_PCODE",
                    projection="mercator"
                   )
fig.update_geos(fitbounds="locations", visible=False)
# fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()


In [46]:
fig2 = px.choropleth_mapbox(df, geojson=indo_states, 
color= 'Kasus_Meni', locations='ADM1_PCODE', featureidkey='properties.ADM1_PCODE',
hover_name= 'Provinsi', hover_data=['Kasus_Posi', 'Kasus_Meni'], 
title='Covid Cases in Indonesia per Province',
mapbox_style = 'carto-positron',
zoom = 3,
opacity=0.5,)

fig2.update_geos(lataxis_showgrid=True, lonaxis_showgrid= True, # shows longitude and latitude lines
fitbounds = 'locations', visible = False # set center of fig to locations of geojson and all other physical attribute(like lakes, rivers etc. to not-visible)
)

fig2.update_layout()

fig2.show()